In [18]:
import json
from typing import List, Dict, Any

def load_loan_data(file_path: str) -> List[Dict[str, Any]]:
    with open(file_path, 'r', encoding='utf-8') as file:
        data_raw = json.load(file)
    data_list = data_raw['result']['baseList']
    data_option = data_raw['result']['optionList']

    new_data = []
    for data in data_list:
        for option in data_option:
            if data["fin_prdt_cd"] == option["fin_prdt_cd"]:
                item_dict = data
                for op_key in option:
                    item_dict[op_key] = option[op_key]
                new_data.append(item_dict)
        
    return data

loan_data = load_loan_data('fss_test.json')

In [ ]:
# # 대출 상품 목록
# loan_data = [
# {'dcls_month': '202410',
#   'fin_co_no': '0010593',
#   'fin_prdt_cd': '302301',
#   'kor_co_nm': '한화생명보험주식회사',
#   'fin_prdt_nm': '홈드림모기지론',
#   'join_way': '영업점,모집인',
#   'loan_inci_expn': '인지세(50%), 주택채권매입비용',
#   'erly_rpay_fee': '기본형(36개월이내 원금상환시 상환금액의 1.2% * 잔존일수 / 36개월) 외 3개 방식',
#   'dly_rate': '대출만기 경과 건을 포함하여 연체기간에 상관없이 정상금리 +3%를 적용 (단, 최고 19%)',
#   'loan_lmt': '감정가의 최고 70%까지 (담보물소재지, 대출금액, 고객신용, 소득 등에 따라 차등적용)',
#   'dcls_strt_day': '20220819',
#   'dcls_end_day': None,
#   'fin_co_subm_day': '202410181400',
#   'mrtg_type': 'E',
#   'mrtg_type_nm': '아파트외',
#   'rpay_type': 'D',
#   'rpay_type_nm': '분할상환방식',
#   'lend_rate_type': 'C',
#   'lend_rate_type_nm': '변동금리',
#   'lend_rate_min': 4.87,
#   'lend_rate_max': 5.37,
#   'lend_rate_avg': 4.76},
# ]

In [ ]:
from langchain.prompts import PromptTemplate

my_template = """아래의 질문에 대해 한 줄로 간결하고 친절하게 답변하세요.
질문: {question}"""

prompt = PromptTemplate.from_template(my_template)
prompt.format(question="잠이 안 올 때는 어떻게 하면 좋을지 대답해줘")

chat_model.predict(prompt.format(question="잠이 안 올 때는 어떻게 하면 좋을지 대답해줘"))

In [20]:
template = """다음 글에서 대출금, 상환 기간, 보유 주택 가격을 찾아 정리하세요.
예시)
질문: 이 상품의 월 상환액을 계산해 주세요. 보유 주택은 3억에 대출금은 1억이고 상환 기간은 36개월입니다.
답변: - 대출금: 1억\n- 상환 기간: 36개월\n- 보유 주택 가격: 3억
질문: {query}"""
prompt = PromptTemplate.from_template(template)
q = "이 상품의 월 상환액을 계산해 주세요. 10억짜리 집이 있고, 2억 정도 대출받아서 12개월동안 갚고 싶어요."

llm.predict(prompt.format(query=q))

'답변: \n- 대출금: 2억\n- 상환 기간: 12개월\n- 보유 주택 가격: 10억'

In [36]:
import json
import re
from typing import List, Dict, Any
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import PromptTemplate
import math

# import openai
# openai.api_key = "your-api-key-here"

# Load loan data from JSON file
def load_loan_data(file_path: str) -> List[Dict[str, Any]]:
    with open(file_path, 'r', encoding='utf-8') as file:
        data_raw = json.load(file)
    data_list = data_raw['result']['baseList']
    data_option = data_raw['result']['optionList']
    new_data = []
    for data in data_list:
        for option in data_option:
            if data["fin_prdt_cd"] == option["fin_prdt_cd"]:
                item_dict = data
                for op_key in option:
                    item_dict[op_key] = option[op_key]
                new_data.append(item_dict)
    return new_data

# Preprocess loan data to create documents
def create_documents(loan_data: List[Dict[str, Any]]) -> List[Document]:
    documents = []
    for product in loan_data:
        content = f"상품명: {product['fin_prdt_nm']}, 회사명: {product['kor_co_nm']}, 최소 연이율: {product.get('lend_rate_min', 'N/A')}, 최대 연이율: {product.get('lend_rate_max', 'N/A')}, 대출 한도: {product['loan_lmt']}, 설명: {product['loan_inci_expn']}"
        documents.append(Document(page_content=content))
    return documents

def extract_user_loan_info(query):
    template = """다음 글에서 대출금, 상환 기간, 보유 주택 가격을 찾아 정리하세요.
    예시)
    질문: 이 상품의 월 상환액을 계산해 주세요. 보유 주택은 3억에 대출금은 1억이고 상환 기간은 3년입니다.
    답변: - 대출금: 100000000\n- 상환 기간: 36개월\n- 보유 주택 가격: 300000000
    질문: {query}"""

    prompt = PromptTemplate.from_template(template)

    extracted_result = llm.predict(prompt.format(query=query))
    # print(extracted_result)

    principal = re.search(r'대출금: (\d+)', extracted_result)
    if not principal:
        return None, None, None
    months = re.search(r'상환 기간: (\d+)', extracted_result)
    house_price = re.search(r'보유 주택 가격: (\d+)', extracted_result)

    principal, months, house_price = map(int, [principal.group(1), months.group(1), house_price.group(1)])
    LTV = house_price / months

    # print(principal, months, LTV)
    return principal, months, LTV

# Calculate monthly repayment amount
def calculate_monthly_repayment(principal: float, annual_rate: float, months: int) -> float:
    monthly_rate = annual_rate / 12 / 100
    if monthly_rate == 0:
        return principal / months
    return principal * (monthly_rate * math.pow(1 + monthly_rate, months)) / (math.pow(1 + monthly_rate, months) - 1)

# Initialize LLM and vector store
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)  # Adjust temperature for response creativity
loan_data = load_loan_data('fss_test.json')
documents = create_documents(loan_data)
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())

# Set up the RetrievalQA chain
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# Define the main query handling function
def handle_user_query(query: str) -> str:
    
    principal, months, LTV = extract_user_loan_info(query)
    if principal and months and LTV:
        query += f"""
        ** 추가 정보 **
        대출금: {principal}, 상환 기간: {months}, LTV: {LTV}
        """

    response = retrieval_qa({"query": query})
    answer = response['result']
    source_docs = response['source_documents']
    
    # Extract relevant information for repayment calculation
    min_annual_rate, max_annual_rate = 0, 0
    for doc in source_docs:
        if '최소 연이율' in doc.page_content:
            min_annual_rate = float(re.search(r'최소 연이율: (\d+.\d+)', doc.page_content).group(1))
        if '최대 연이율' in doc.page_content:
            max_annual_rate = float(re.search(r'최대 연이율: (\d+.\d+)', doc.page_content).group(1))
    
    if principal is not None:
        # answer = ""
        if min_annual_rate:
            monthly_repayment = calculate_monthly_repayment(principal, min_annual_rate, months)
            answer += f"\n\n월 상환액(최소): {monthly_repayment:.2f}원"
        if max_annual_rate:
            monthly_repayment = calculate_monthly_repayment(principal, max_annual_rate, months)
            answer += f"\n월 상환액(최대): {monthly_repayment:.2f}원"
    
    # Optionally format the output to include source documents
    source_texts = "\n".join([doc.page_content for doc in source_docs])
    # return f"{answer}\n\n**참조 문서:**\n{source_texts}
    return answer

# Example user queries
queries = [
    "주택담보대출의 이자율은 얼마인가요?",
    "가장 낮은 연이율의 상품을 추천해 주세요.",
    "이 상품의 월 상환액을 계산해 주세요. 보유 주택은 3억에 대출금은 1억이고 상환 기간은 36개월입니다.",
]

# Execute queries
for query in queries:
    response = handle_user_query(query)
    print(f"질문: {query}\n응답: {response}\n")


질문: 주택담보대출의 이자율은 얼마인가요?
응답: 주택담보대출의 이자율은 최소 4.58%에서 최대 5.85%까지 변동합니다.

질문: 가장 낮은 연이율의 상품을 추천해 주세요.
응답: 홈드림모기지론 상품의 최소 연이율은 4.87%입니다.

100000000 36 8333333.333333333
질문: 이 상품의 월 상환액을 계산해 주세요. 보유 주택은 3억에 대출금은 1억이고 상환 기간은 36개월입니다.
응답: 

월 상환액(최소): 2972458.40원
월 상환액(최대): 3006527.34원



In [12]:
response.page_content

'상품명: 홈드림모기지론, 회사명: 한화생명보험주식회사, 최소 연이율: 4.87, 최대 연이율: 5.37, 대출 한도: 감정가의 최고 70%까지 (담보물소재지, 대출금액, 고객신용, 소득 등에 따라 차등적용), 설명: 인지세(50%), 주택채권매입비용'

In [32]:
re.search(r'보유 주택 가격: (\d+)', """답변: 
- 대출금: 1억
- 상환 기간: 36개월
- 보유 주택 가격: 3억""").group(1)

'3'

In [31]:
loan_data

{'dcls_month': '202410',
 'fin_co_no': '0013174',
 'fin_prdt_cd': 'WR0001D',
 'kor_co_nm': '농협손해보험주식회사',
 'fin_prdt_nm': '헤아림아파트론Ⅰ',
 'join_way': '영업점',
 'loan_inci_expn': '- 인지세 : 해당 세액의 50%\n- 국민주택채권매입비\n- 근저당권 말소비용',
 'erly_rpay_fee': '- 중도상환원금 X 1.2%(변동), 1.4%(고정) X (잔여기간/대출기간)\n- 대출일로부터 1년 경과후 최초대출금액의 50%까지 면제\n- 대출일로부터 3년 경과시 전액 면제',
 'dly_rate': '- 연체기간에 관계없이 연체일수 X (채무자 대출금리 + 3%)÷365(윤년은 366)\n- 지연배상금률이 연 17%를 초과하는 경우에는 연 17%를 적용',
 'loan_lmt': 'LTV 30% ~ 70% \n - 지역별 차등적용',
 'dcls_strt_day': '20241017',
 'dcls_end_day': None,
 'fin_co_subm_day': '202410171600',
 'mrtg_type': 'A',
 'mrtg_type_nm': '아파트',
 'rpay_type': 'D',
 'rpay_type_nm': '분할상환방식',
 'lend_rate_type': 'C',
 'lend_rate_type_nm': '변동금리',
 'lend_rate_min': 3.79,
 'lend_rate_max': 6.3,
 'lend_rate_avg': 3.77}

In [6]:
# OpenAI Embeddings를 통해 임베딩 생성
embeddings = OpenAIEmbeddings()

# FAISS 벡터스토어에 문서들을 인덱싱
vectorstore = FAISS.from_documents(documents, embeddings)


In [7]:
def calculate_monthly_repayment(principal, annual_rate, years):
    monthly_rate = annual_rate / 12 / 100
    months = years * 12
    monthly_payment = principal * monthly_rate * (1 + monthly_rate)**months / ((1 + monthly_rate)**months - 1)
    return round(monthly_payment, 2)


In [9]:
# 사용자 정의 프롬프트 템플릿 설정
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="사용자가 문의한 내용에 맞는 정보를 제공해 주세요:\n\n{context}\n\n질문: {question}"
)

# LLM과 프롬프트를 이용한 QA 체인 구성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt_template)

# # RAG Chain 구성
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# qa_chain = RetrievalQA.from_chain_type(
#     llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.3),
#     chain_type="stuff",
#     retriever=retriever,
#     prompt=prompt_template
# )

# 사용자가 입력한 질문을 처리하는 함수
def handle_user_query(query, principal=None, annual_rate=None, years=None):
    # 대출 계산 질문이 있는 경우 월 상환액 계산 수행
    if "상환액" in query and principal and annual_rate and years:
        monthly_payment = calculate_monthly_repayment(principal, annual_rate, years)
        return f"대출금 {principal}원에 대한 월 상환액은 {monthly_payment}원입니다. 기간: {years}년, 이자율: {annual_rate}%"

    # RAG 시스템을 통해 일반적인 금융 질문 답변 생성
    response = qa_chain({"query": query})
    return response["result"]

# 예시 질문과 계산 수행
query = "이 상품의 상환액을 계산해주세요"
response = handle_user_query(query, principal=100000000, annual_rate=3.41, years=20)
print(response)


대출금 100000000원에 대한 월 상환액은 575345.67원입니다. 기간: 20년, 이자율: 3.41%


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13472\1442227940.py:9: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt_template)
